I would like to look at some a very simple way of mapping data in python to a musical output.


In [4]:
from pynput import mouse
import time
import numpy as np

last_pos = None
last_time = None
gesture_energy = 0.0

def on_move(x, y):
    global last_pos, last_time, gesture_energy

    now = time.time()
    if last_pos is not None:
        dx = x - last_pos[0]
        dy = y - last_pos[1]
        dt = now - last_time

        if dt > 0:
            velocity = np.sqrt(dx*dx + dy*dy) / dt
            gesture_energy = 0.9 * gesture_energy + 0.1 * velocity

    last_pos = (x, y)
    last_time = now

listener = mouse.Listener(on_move=on_move)
listener.start()


In [2]:
import sounddevice as sd

fs = 44100
block_size = 256

def audio_callback(outdata, frames, time, status):
    global gesture_energy
    t = np.arange(frames) / fs
    signal = np.sin(2 * np.pi * 220 * t)

    outdata[:, 0] = signal * 0.1
    outdata[:, 1] = signal * 0.1

with sd.OutputStream(channels=2, callback=audio_callback,
                     samplerate=fs, blocksize=block_size):
    while True:
        time.sleep(0.1)


KeyboardInterrupt: 